In [2]:
import BioSimSpace.Sandpit.Exscientia as BSS
import numpy as np
import pandas as pd

INFO:rdkit:Enabling RDKit 2024.03.5 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [5]:
system = BSS.IO.readMolecules(["bound/ensemble_equilibration_1/bound_preequil.prm7","bound/ensemble_equilibration_1/bound_preequil.rst7"])

In [3]:
lig = BSS.Align.decouple(system[0])
system.updateMolecule(0, lig)

In [5]:
system[0].getResidues()

[<BioSimSpace.Residue: name='LIG', molecule=45, index=0, nAtoms=97>]

In [4]:
traj = BSS.Trajectory.Trajectory(
    trajectory="bound/ensemble_equilibration_1/gromacs.xtc",
    topology="bound/ensemble_equilibration_1/bound_preequil.prm7",
)


Indexing XTC                15 s 92540240.0 bytes / s          


In [6]:
restraint = BSS.FreeEnergy.RestraintSearch.analyse(
    work_dir = "./get_restraints",
    system = system,
    traj = traj,
    temperature = 298 * BSS.Units.Temperature.kelvin,
    method="BSS",
    restraint_type = "multiple_distance",
)

INFO:MDAnalysis.core.universe:The attribute(s) masses, types have already been read from the topology file. The guesser will only guess empty values for this attribute, if any exists. To overwrite it by completely guessed values, you can pass the attribute to the force_guess parameter instead of the to_guess one
INFO:MDAnalysis.guesser.base:There is no empty types values. Guesser did not guess any new values for types attribute
INFO:MDAnalysis.guesser.base:There is no empty masses values. Guesser did not guess any new values for masses attribute


Searching for low variance pairs. Frame no:   0%|          | 0/2501 [00:00<?, ?it/s]

In [7]:
restrain_dict=restraint.toString(engine="Somd")

In [11]:
restraint.getCorrection(method="numerical")

/home/yanfz/miniconda3/envs/openbiosim/lib/python3.10/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_restraint.py:1157: UserWarning: The multiple distance restraint correction is assumes that only the 'permanent_distance_restraint' is active.
  _warnings.warn(


-0.6672 kcal/mol

In [8]:
lam_vals_release_restraint = [0.000, 0.100, 0.200, 0.300, 0.400, 0.500, 0.600, 0.700, 0.800, 0.900, 1.000]
for i in range (1,4):

    release_restraints_protocol = BSS.Protocol.FreeEnergy(runtime=1*BSS.Units.Time.nanosecond, 
                                                      timestep=4*BSS.Units.Time.femtosecond, 
                                                      lam_vals=lam_vals_release_restraint, 
                                                      perturbation_type="release_restraint")

    release_restraints_fe_calc = BSS.FreeEnergy.AlchemicalFreeEnergy(restraint.system, 
                                                                 release_restraints_protocol, 
                                                                 engine='somd', 
                                                                 restraint=restraint,
                                                                 work_dir=f'./get_restraints/release{i}')

/home/yanfz/miniconda3/envs/openbiosim/lib/python3.10/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_alchemical_free_energy.py:370: UserWarning: SOMD simulations with some non-interacting ligands can be unstable. This affects some systems with multiple distance restraints during the release restraint state. If you experience problems, consider using multiple distance restraints with GROMACS instead.
  _warnings.warn(
/home/yanfz/miniconda3/envs/openbiosim/lib/python3.10/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_alchemical_free_energy.py:370: UserWarning: SOMD simulations with some non-interacting ligands can be unstable. This affects some systems with multiple distance restraints during the release restraint state. If you experience problems, consider using multiple distance restraints with GROMACS instead.
  _warnings.warn(
/home/yanfz/miniconda3/envs/openbiosim/lib/python3.10/site-packages/BioSimSpace/Sandpit/Exscientia/FreeEnergy/_alchemical_free_energy.py:3